In [1]:
# !mkdir diff-run

하위 디렉터리 또는 파일 diff-run이(가) 이미 있습니다.


In [2]:
# !mkdir diff-run/images

In [2]:
# torch summary https://honeyjamtech.tistory.com/40

명령 구문이 올바르지 않습니다.


In [1]:
import torch
import numpy as np
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable
from torchvision.utils import save_image
from torchvision.utils import make_grid
from torch.utils.tensorboard import SummaryWriter
from torchsummary import summary
import datetime

In [2]:
torch.manual_seed(1)

In [3]:
writer = SummaryWriter('diff-run/py-gan')

In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
batch_size = 128

In [5]:
train_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize([0.5], [0.5])])
train_dataset = datasets.FashionMNIST(root='./data/', train=True, transform=train_transform, download=True)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)

  0%|          | 0/26421880 [00:00<?, ?it/s]

Extracting ./data/FashionMNIST\raw\train-images-idx3-ubyte.gz to ./data/FashionMNIST\raw



  0%|          | 0/29515 [00:00<?, ?it/s]

Extracting ./data/FashionMNIST\raw\train-labels-idx1-ubyte.gz to ./data/FashionMNIST\raw



  0%|          | 0/4422102 [00:00<?, ?it/s]

Extracting ./data/FashionMNIST\raw\t10k-images-idx3-ubyte.gz to ./data/FashionMNIST\raw



  0%|          | 0/5148 [00:00<?, ?it/s]

Extracting ./data/FashionMNIST\raw\t10k-labels-idx1-ubyte.gz to ./data/FashionMNIST\raw

Processing...
Done!


C:\Users\user\anaconda3\envs\sdf\lib\site-packages\torchvision\datasets\mnist.py:502: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  ..\torch\csrc\utils\tensor_numpy.cpp:143.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [6]:
image_shape = (1, 28, 28)
image_dim = int(np.prod(image_shape))
latent_dim = 100

In [7]:
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()

        self.model = nn.Sequential(nn.Linear(latent_dim, 128),
                                    nn.LeakyReLU(0.2, inplace=True),
                                    nn.Linear(128, 256),
                      nn.BatchNorm1d(256, 0.8),
                                    nn.LeakyReLU(0.2, inplace=True), 
                                    nn.Linear(256, 512),
                      nn.BatchNorm1d(512, 0.8),
                                    nn.LeakyReLU(0.2, inplace=True), 
                      nn.Linear(512, 1024),
                      nn.BatchNorm1d(1024, 0.8),
                                    nn.LeakyReLU(0.2, inplace=True), 
                                    nn.Linear(1024, image_dim),
                                    nn.Tanh())

    def forward(self, noise_vector): 
        image = self.model(noise_vector)
        image = image.view(image.size(0), *image_shape)
        return image

In [8]:
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()

        self.model = nn.Sequential(nn.Linear(image_dim, 512),
                                    nn.LeakyReLU(0.2, inplace=True),
                                    nn.Linear(512, 256),
                                    nn.LeakyReLU(0.2, inplace=True),
                                    nn.Linear(256, 1),
                                    nn.Sigmoid())

    def forward(self, image):
        image_flattened = image.view(image.size(0), -1)
        result = self.model(image_flattened)
        return result     

In [9]:
generator = Generator().to(device)
discriminator = Discriminator().to(device)

In [ ]:
#torch.save(generator.state_dict(), 'generator.pth')

In [ ]:
# for layer in generator.children():
#     print(layer.type)

In [10]:
summary(generator, (100,))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1                  [-1, 128]          12,928
         LeakyReLU-2                  [-1, 128]               0
            Linear-3                  [-1, 256]          33,024
       BatchNorm1d-4                  [-1, 256]             512
         LeakyReLU-5                  [-1, 256]               0
            Linear-6                  [-1, 512]         131,584
       BatchNorm1d-7                  [-1, 512]           1,024
         LeakyReLU-8                  [-1, 512]               0
            Linear-9                 [-1, 1024]         525,312
      BatchNorm1d-10                 [-1, 1024]           2,048
        LeakyReLU-11                 [-1, 1024]               0
           Linear-12                  [-1, 784]         803,600
             Tanh-13                  [-1, 784]               0
Total params: 1,510,032
Trainable param

In [11]:
summary(discriminator, (1,28,28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1                  [-1, 512]         401,920
         LeakyReLU-2                  [-1, 512]               0
            Linear-3                  [-1, 256]         131,328
         LeakyReLU-4                  [-1, 256]               0
            Linear-5                    [-1, 1]             257
           Sigmoid-6                    [-1, 1]               0
Total params: 533,505
Trainable params: 533,505
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.01
Params size (MB): 2.04
Estimated Total Size (MB): 2.05
----------------------------------------------------------------


In [12]:
adversarial_loss = nn.BCELoss() 

In [13]:
learning_rate = 0.0002 
G_optimizer = optim.Adam(generator.parameters(), lr = learning_rate, betas=(0.5, 0.999))
D_optimizer = optim.Adam(discriminator.parameters(), lr = learning_rate, betas=(0.5, 0.999))

In [14]:
cuda = True if torch.cuda.is_available() else False
Tensor = torch.cuda.FloatTensor if cuda else torch.FloatTensor

In [15]:
num_epochs = 500
D_loss_plot, G_loss_plot = [], []
for epoch in range(1, num_epochs+1): 

    D_loss_list, G_loss_list = [], []
   
    for index, (real_images, _) in enumerate(train_loader):
        D_optimizer.zero_grad()
        real_images = real_images.to(device)
        real_target = Variable(torch.ones(real_images.size(0), 1).to(device))
        fake_target = Variable(torch.zeros(real_images.size(0), 1).to(device))

        D_real_loss = adversarial_loss(discriminator(real_images), real_target)
        # print(discriminator(real_images))

        noise_vector = Variable(torch.randn(real_images.size(0), latent_dim).to(device))
        #noise_vector = Variable(Tensor(np.random.normal(0, 1, \
        #                                                (real_images.size(0),\
        #                                                 latent_dim))))
        noise_vector = noise_vector.to(device)
        generated_image = generator(noise_vector)

        D_fake_loss = adversarial_loss(discriminator(generated_image),\
                                     fake_target)

        D_total_loss = D_real_loss + D_fake_loss
        D_loss_list.append(D_total_loss)
        D_total_loss.backward()
        D_optimizer.step()

        G_optimizer.zero_grad()
        generated_image = generator(noise_vector)
        G_loss = adversarial_loss(discriminator(generated_image), real_target)
        G_loss_list.append(G_loss)

        G_loss.backward()
        G_optimizer.step()
        d = generated_image.data

        writer.add_scalar('Discriminator Loss',
                            D_total_loss,
                            epoch * len(train_loader) + index)

        writer.add_scalar('Generator Loss',
                            G_loss,
                            epoch * len(train_loader) + index)


    print('Epoch: [%d/%d]: D_loss: %.3f, G_loss: %.3f' % (
            (epoch), num_epochs, torch.mean(torch.FloatTensor(D_loss_list)),\
             torch.mean(torch.FloatTensor(G_loss_list))))

    D_loss_plot.append(torch.mean(torch.FloatTensor(D_loss_list)))
    G_loss_plot.append(torch.mean(torch.FloatTensor(G_loss_list)))
    save_image(generated_image.data[:90], 'diff-run/images/sample_%d'%epoch + '.png', nrow=10, normalize=True)

Epoch: [1/500]: D_loss: 1.002, G_loss: 1.380
Epoch: [2/500]: D_loss: 0.930, G_loss: 1.932
Epoch: [3/500]: D_loss: 1.037, G_loss: 1.584
Epoch: [4/500]: D_loss: 1.128, G_loss: 1.332
Epoch: [5/500]: D_loss: 1.165, G_loss: 1.275
Epoch: [6/500]: D_loss: 1.150, G_loss: 1.235
Epoch: [7/500]: D_loss: 1.177, G_loss: 1.226
Epoch: [8/500]: D_loss: 1.188, G_loss: 1.200
Epoch: [9/500]: D_loss: 1.227, G_loss: 1.107
Epoch: [10/500]: D_loss: 1.228, G_loss: 1.093
Epoch: [11/500]: D_loss: 1.231, G_loss: 1.085
Epoch: [12/500]: D_loss: 1.237, G_loss: 1.080
Epoch: [13/500]: D_loss: 1.234, G_loss: 1.080
Epoch: [14/500]: D_loss: 1.248, G_loss: 1.044
Epoch: [15/500]: D_loss: 1.251, G_loss: 1.021
Epoch: [16/500]: D_loss: 1.257, G_loss: 1.034
Epoch: [17/500]: D_loss: 1.256, G_loss: 1.021
Epoch: [18/500]: D_loss: 1.267, G_loss: 0.991
Epoch: [19/500]: D_loss: 1.265, G_loss: 0.976


KeyboardInterrupt: 